In [35]:
import pandas as pd
df = pd.read_csv("data2.csv").drop(columns=['Agency Name (Crash Level) ', 'Area: County',
       'Area: City', 'Roadway (From Crash Report)','Manner of Collision (Crash Level) ', 'SHSP Emphasis Areas',
       '# of Fatalities per Crash', '# Serious Injuries', '# Visible Injuries',
       '# Complaint Injuries', '# of Vehicles per crash ']).dropna(
    subset=['Intersection Name (from Crash Report)'], how='all')

In [36]:
dfCandice = df.head(20000).copy()
dfPaul = df.iloc[20000:40001].copy()
dfMac = df.iloc[40001:].copy()

In [37]:
from arcgis.geocoding import geocode, get_geocoders
from arcgis.gis import GIS

################## INSERT API KEY HERE ##################
gis = GIS(api_key='AAPKa243c48bd15242a78b2d533ba71125645TiI2vGrnQZMNkGMtGeB4RTRZoKLieDI0Uk0zkFvnle-h1OAiQ3WPeFKTwGMLela')

In [38]:
def getCoords(s):
    k = geocode(s + ', Fulton County, GA')
    try:
        k=k[0]['location']
        return k['x'], k['y']
    except:
        return 0,0

In [39]:
############### Replace these with your dataframe variable name (both lines) #########################
dfRun = dfMac
output = f'{dfMac=}'.split('=')[0]

In [40]:
# This is the long step
dfRun['lat'], dfRun['lng'] = zip(*dfRun['Intersection Name (from Crash Report)'].apply(getCoords))
dfRun.drop(columns = ['Intersection Name (from Crash Report)']).to_csv(output + '2020.csv')

In [41]:
######### Data Cleaning ###########

In [111]:
import os

filepaths = [('csv/' + f) for f in os.listdir("./csv") if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))

df['Date and Time'] = pd.to_datetime(df['Date and Time'])
df['Season'] = df.apply(
    lambda r: r['Date and Time'].month%12 // 3, axis=1
).replace({0:'Winter', 1:'Spring', 2:'Summer', 3:'Fall'})
df['TODBucket'] = pd.cut(
    df['Date and Time'].dt.hour,
    [0,6,7,9,15.5,19,20,24],    # TOD Buckets
    labels=['Night', 'Day', 'RH', 'Day', 'RH', 'Day', 'Night'],
    right=False, 
    ordered=False
)
df = df.rename(columns={
    'KABCO Severity': 'Outcome',
    'Weather Conditions (Crash Level)': 'Weather',
    'Surface Condition (Crash Level) ': 'Surface',
    'Light Conditions (Crash Level)': 'Light'
})

In [114]:
df['DOW'] = df['Date and Time'].dt.dayofweek
df.loc[df['TODBucket'].eq('RH') & (df['DOW'].eq(5) | df['DOW'].eq(6)), 'TODBucket'] = 'Day'

In [166]:
dlsMask = (
    ((df['Date and Time'] > pd.Timestamp(2020, 3, 8)) & (df['Date and Time'] <= pd.Timestamp(2020, 3, 10))) |
    ((df['Date and Time'] > pd.Timestamp(2020, 11, 1)) & (df['Date and Time'] <= pd.Timestamp(2020, 11, 3))) |
    ((df['Date and Time'] > pd.Timestamp(2021, 3, 14)) & (df['Date and Time'] <= pd.Timestamp(2021, 3, 15))) |
    ((df['Date and Time'] > pd.Timestamp(2021, 11, 7)) & (df['Date and Time'] <= pd.Timestamp(2021, 11, 9)))
)

holidayMask = (
    ((df['Date and Time'] > pd.Timestamp(2020, 12, 24)) & (df['Date and Time'] <= pd.Timestamp(2020, 3, 26))) | #XMAS
    ((df['Date and Time'] > pd.Timestamp(2021, 12, 24)) & (df['Date and Time'] <= pd.Timestamp(2021, 3, 26))) | #XMAS
    ((df['Date and Time'] > pd.Timestamp(2019, 12, 24)) & (df['Date and Time'] <= pd.Timestamp(2020, 1, 2))) | #NY
    ((df['Date and Time'] > pd.Timestamp(2020, 12, 24)) & (df['Date and Time'] <= pd.Timestamp(2021, 1, 2))) | #NY
    ((df['Date and Time'] > pd.Timestamp(2021, 12, 24)) & (df['Date and Time'] <= pd.Timestamp(2022, 1, 2))) | #NY
    ((df['Date and Time'] > pd.Timestamp(2020, 5, 5)) & (df['Date and Time'] <= pd.Timestamp(2020, 5, 6))) | #CDM
    ((df['Date and Time'] > pd.Timestamp(2021, 5, 5)) & (df['Date and Time'] <= pd.Timestamp(2021, 5, 6))) | #CDM
    ((df['Date and Time'] > pd.Timestamp(2020, 7, 4)) & (df['Date and Time'] <= pd.Timestamp(2020, 7, 5))) | #J4
    ((df['Date and Time'] > pd.Timestamp(2021, 7, 4)) & (df['Date and Time'] <= pd.Timestamp(2021, 7, 5))) | #J4
    ((df['Date and Time'] > pd.Timestamp(2020, 3, 17)) & (df['Date and Time'] <= pd.Timestamp(2020, 3, 18))) | #SPD
    ((df['Date and Time'] > pd.Timestamp(2021, 3, 17)) & (df['Date and Time'] <= pd.Timestamp(2021, 3, 18))) #SPD    
)

In [167]:
df['DLS'] = dlsMask.astype(int)
df['Holiday'] = holidayMask.astype(int)

In [174]:
df.drop(df.columns[0:2], axis=1).to_csv('accData.csv')

In [175]:
df

,Unnamed: 0,Date and Time,Outcome,Weather,Surface,Light,lat,lng,Season,TODBucket,DOW,DLS,Holiday
0,0,2020-10-11 02:20:00,(O) No Injury,Rain,Wet,Dark-Lighted,-84.36166,33.81943,Fall,Night,6,0,0
1,1,2020-11-30 18:17:00,(O) No Injury,Rain,Wet,Dark-Lighted,-84.34311,34.03851,Fall,RH,0,0,0
2,2,2020-11-25 09:00:00,(O) No Injury,Clear,Dry,Daylight,-84.57278,33.72991,Fall,Day,2,0,0
3,3,2020-09-29 05:48:00,(O) No Injury,Cloudy,Wet,Dark-Not Lighted,-84.40855,33.66848,Fall,Night,1,0,0
4,4,2020-09-15 01:26:00,(O) No Injury,Clear,Dry,Dark-Lighted,-84.35708,33.82307,Fall,Night,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,43465,2020-02-20 08:01:00,(C) Possible Injury / Complaint,Rain,Wet,Daylight,-84.26860,34.07476,Winter,RH,3,0,0
19997,43466,2020-02-23 12:19:00,(O) No Injury,Cloudy,Dry,Daylight,-84.44356,33.67036,Winter,Day,6,0,0
19998,43467,2020-01-11 21:49:00,(O) No Injury,Clear,Dry,Dark-Lighted,-84.39034,33.76468,Winter,Night,5,0,0
19999,43468,2020-08-24 17:21:00,(B) Suspected Minor/Visible Injury,Clear,Dry,Daylight,-84.40163,33.73702,Summer,RH,0,0,0
